In [2]:
import urllib
import urllib.request
import json
import pandas as pd
import math

#eSTATにユーザー登録して取得したアプリケーションIDを設定します
appId = "d3e44dcd0cfa297acc6d0553213252e8778f4454"

#00000202~~は基礎項目データ、000002030~~は社会生活統計指標項目のstatsDataIdです。
Datalist = ["0000020201", "0000020202", "0000020203", "0000020204", "0000020205", "0000020206", 
            "0000020207", "0000020208", "0000020209", "0000020210", "0000020211",
            "0000020301", "0000020302", "0000020303", "0000020304", "0000020305", "0000020306", 
            "0000020307", "0000020308", "0000020309", "0000020310", "0000020311"]


url = "https://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?"
for dataID in Datalist:
    #設定するキー : 2014年以降のデータをとる
    keys = {
        "appId"            : appId,
        "lang"             : "J" ,
        "statsDataId"      : dataID ,
        "metaGetFlg"       : "Y" ,
        "cntGetFlg"        : "N",
        "sectionHeaderFlg" : "1",
        "cdTimeFrom"       : "2014", 
    }
    #keysからパラメータの文字列を生成
    paramStr = urllib.parse.urlencode(keys)
    #JSONデータの取得
    rObj   = urllib.request.urlopen(url + paramStr)
    resStr = rObj.read()
    #json形式として読み込む
    res = json.loads(resStr)
    #Json形式で保存
    with open('./DL_json/' + dataID + '_1.json', 'w') as f:
        json.dump(res, f, ensure_ascii=False, indent=None)
    
    # 前回取り込んだデータの情報からNEXT_KEYがある場合は取り出して変数に入れてforループ
    if res["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]["TOTAL_NUMBER"] > 100000:
        next_key = res["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]["NEXT_KEY"]
        #１回目に取り込んだデータの情報からforで繰り返す回数を計算して　x　に入れる
        x = math.floor(res["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]["TOTAL_NUMBER"]/100000)
        for i in range(x):
            keys = {
                "appId"            : appId,
                "lang"             : "J" ,
                "statsDataId"      : dataID ,
                "metaGetFlg"       : "Y" ,
                "cntGetFlg"        : "N",
                "sectionHeaderFlg" : "1",
                "cdTimeFrom"       : "2014",
                "startPosition"    : next_key,
                }
            #keysからパラメータの文字列を生成
            paramStr = urllib.parse.urlencode(keys)
            #JSONデータの取得
            rObj   = urllib.request.urlopen(url + paramStr)
            resStr = rObj.read()
            #json形式として読み込む
            res = json.loads(resStr)
            #Json形式で保存
            name = str(i+2)
            with open('./DL_json/' + dataID + '_' + name + '.json', 'w') as f:
                json.dump(res, f, ensure_ascii=False, indent=None)

In [5]:
import json
import pandas as pd
import math

# 社会生活統計指標項目のみでCSVを作る
# マージするJsonを全部読み込む
dfAs_1 = pd.read_json("./DL_json/0000020301_1.json", encoding="UTF-8")
dfBs_1 = pd.read_json("./DL_json/0000020302_1.json", encoding="UTF-8")
dfCs_1 = pd.read_json("./DL_json/0000020303_1.json", encoding="UTF-8")
dfDs_1 = pd.read_json("./DL_json/0000020304_1.json", encoding="UTF-8")
dfEs_1 = pd.read_json("./DL_json/0000020305_1.json", encoding="UTF-8")
dfFs_1 = pd.read_json("./DL_json/0000020306_1.json", encoding="UTF-8")
dfGs_1 = pd.read_json("./DL_json/0000020307_1.json", encoding="UTF-8")
dfHs_1 = pd.read_json("./DL_json/0000020308_1.json", encoding="UTF-8")
dfIs_1 = pd.read_json("./DL_json/0000020309_1.json", encoding="UTF-8")
dfJs_1 = pd.read_json("./DL_json/0000020310_1.json", encoding="UTF-8")

df_As = pd.DataFrame(dfAs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Bs = pd.DataFrame(dfBs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Cs = pd.DataFrame(dfCs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Ds = pd.DataFrame(dfDs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Es = pd.DataFrame(dfEs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Fs = pd.DataFrame(dfFs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Gs = pd.DataFrame(dfGs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Hs = pd.DataFrame(dfHs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Is = pd.DataFrame(dfIs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])
df_Js = pd.DataFrame(dfJs_1["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])

df_all = df_As.append([df_Bs, df_Cs, df_Ds, df_Es, df_Fs, df_Gs, df_Hs, df_Is, df_Js])

df_all2 = df_all.replace(["-"], "NaN")
df_all2["value"]=df_all2["$"].astype(float)
df_all_pivot = df_all2.pivot_table(values=['value'], index=['@area'], columns=['@cat01'])

df_all_pivot.to_csv('./csv/Social_life_statistics_index.csv', encoding='CP932')